<center><img src="Logo PI MLOps STEAM.png" alt="Logo PI MLOps STEAM"></center>

In [1]:
# Importo librerías.
import json
import pandas as pd
import ast
import gzip
from tqdm import tqdm

# Creo la función para leer las líneas de cada dataset.
def leer_datos(ruta, tipo='json'):
    """
    Esta función lee datos desde cada archivo de mi carpeta y los convierte en un DataFrame de pandas.

    Args:
        ruta (str): La ruta al archivo que se va a leer.
        tipo (str, optional): El formato de los datos en el archivo. Puede ser 'json' o 'literal'. 
                              Si es 'json', los datos se cargarán utilizando json.loads. 
                              Si es 'literal', los datos se cargarán utilizando ast.literal_eval. 
                              Por defecto es 'json'.

    Returns:
        pd.DataFrame: Un DataFrame de pandas que contiene los datos leídos del archivo.
    """
    filas = []  # Será la lista para almacenar cada fila de datos

    # Abro el archivo en modo lectura con codificación utf-8
    with gzip.open(ruta, 'rt', encoding='utf-8') as f:
        # Leo cada línea del archivo
        for line in tqdm(f.readlines(), desc=f"Leyendo {ruta}"):
            # Si el tipo es 'json', carga los datos con json.loads
            if tipo == 'json':
                data = json.loads(line)
            # Si el tipo es 'literal', carga los datos con ast.literal_eval
            elif tipo == 'literal':
                data = ast.literal_eval(line)
            # Añado los datos a la lista de filas
            filas.append(data)

    # Convierto la lista de filas en un DataFrame de pandas y lo devuelve.
    return pd.DataFrame(filas)

# Determino la ruta de cada dataset.
ruta_games = r'C:\Users\Esteban García\OneDrive\Escritorio\LABs\PI MLOps STEAM DTPT06 01\Datos\steam_games.json.gz'
ruta_reviews = r'C:\Users\Esteban García\OneDrive\Escritorio\LABs\PI MLOps STEAM DTPT06 01\Datos\user_reviews.json.gz'
ruta_items = r'C:\Users\Esteban García\OneDrive\Escritorio\LABs\PI MLOps STEAM DTPT06 01\Datos\users_items.json.gz'

# Convierto en dataframe cada datasets
df_steam_games = leer_datos(ruta_games, tipo='json')
df_user_reviews = leer_datos(ruta_reviews, tipo='literal')
df_users_items = leer_datos(ruta_items, tipo='literal')


Leyendo C:\Users\Esteban García\OneDrive\Escritorio\LABs\PI MLOps STEAM DTPT06 01\Datos\steam_games.json.gz: 100%|██████████| 120445/120445 [00:01<00:00, 79186.92it/s]
Leyendo C:\Users\Esteban García\OneDrive\Escritorio\LABs\PI MLOps STEAM DTPT06 01\Datos\user_reviews.json.gz: 100%|██████████| 25799/25799 [00:03<00:00, 6950.12it/s]
Leyendo C:\Users\Esteban García\OneDrive\Escritorio\LABs\PI MLOps STEAM DTPT06 01\Datos\users_items.json.gz: 100%|██████████| 88310/88310 [03:51<00:00, 380.92it/s] 


<h3 style="color:green">
Nuestros dataframes se llaman: df_steam_games; df_user_reviews y df_users_items

<h5 style="color:blue">
Trabajaré son Steam Games.

### Verifico

In [ ]:
# Visualizo steam
df_steam_games

### Verifico las columnas que contienen listas o diccionarios.

In [ ]:
# Importo librerías.
import pandas as pd

# Itero sobre cada columna
for column in df_steam_games.columns:
    # Obtengo el primer valor no nulo
    non_null_values = df_steam_games[column].dropna()
    if non_null_values.empty:
        print(f"La columna '{column}' solo contiene valores nulos.")
        continue
    first_non_null_value = non_null_values.iloc[0]

    # Compruebo si es una lista o un diccionario
    if isinstance(first_non_null_value, list):
        print(f"La columna '{column}' contiene una lista.")
    elif isinstance(first_non_null_value, dict):
        print(f"La columna '{column}' contiene un diccionario.")
    else:
        print(f"La columna '{column}' no contiene ni listas ni diccionarios.")

### Elimino duplicados.

La finalidad es eliminar los que realmente están duplicados, por eso selecciono un porcentaje y lo comparo antes de eliminarlo, de esa forma me aseguro que estoy eliminando lo que efectivamente está duplicado.

In [ ]:
# Importo librerías.
import numpy as np

# Obtengo todas las columnas del DataFrame
all_columns = df_steam_games.columns.tolist()

# Columnas que contienen listas
list_columns = ['genres', 'tags', 'specs']

# Elimino las columnas que contienen listas de all_columns
all_columns = [col for col in all_columns if col not in list_columns]

# Aseguro que 'developer' esté en la lista de columnas a verificar
if 'developer' not in all_columns:
    all_columns.append('developer')

# Calculo el número de columnas a seleccionar (30% de todas las columnas)
num_columns_to_check = int(len(all_columns) * 0.3)

# Selecciono un subconjunto de columnas al azar
np.random.seed(0)  # Para reproducibilidad
columns_to_check = np.random.choice(all_columns, num_columns_to_check, replace=False)

# Convierto la lista en una tupla
columns_to_check = tuple(columns_to_check)

# Imprimo el número de filas antes de la eliminación de duplicados
print("Número de filas antes de la eliminación de duplicados:", df_steam_games.shape[0])

# Elimino los duplicados del DataFrame
df_steam_games.drop_duplicates(subset=columns_to_check, keep='first', inplace=True)

# Imprimo el número de filas después de la eliminación de duplicados
print("Número de filas después de la eliminación de duplicados:", df_steam_games.shape[0])

# Imprimo las columnas que se usaron para verificar los duplicados
print("\nColumnas que se usaron para verificar los duplicados:") 
print(columns_to_check[:10])


Lo que hicimos fue eliminar duplicados considerando las columnas: 'genres', 'tags' y 'specs' que contienen una lista.

### A continuación normalizaremos.

#### Normalizo mi df_steam_games eliminando filas vacías, NaN y analizaremos en porcentajes cuántos datos nulos o vacíos representan en el total del df y desanidando 'genres'.

In [ ]:
# Importo librerías.
import pandas as pd

# Guardo el número de filas antes de la eliminación.
num_rows_before = df_steam_games.shape[0]

# Imprimo los tipos de datos por columna y cantidad de nulos.
print("Tipos de datos por columna y cantidad de nulos:")
print(df_steam_games.dtypes)
print(df_steam_games.isnull().sum())

# Elimino las filas donde cualquier valor es NaN
print("Eliminando filas con valores nulos...")
df_steam_games = df_steam_games.dropna(how='any').reset_index(drop=True)

# Reseteo el índice del DataFrame
print("Reseteando el índice del DataFrame...")
df_steam_games.reset_index(drop=True, inplace=True)

# Guardo el número de filas después de la eliminación
num_rows_after = df_steam_games.shape[0]

# Calculo la reducción porcentual
reduction_percentage = (1 - num_rows_after / num_rows_before) * 100

# Imprimo la reducción porcentual
print(f"La reducción de filas es aproximadamente del {reduction_percentage:.2f}%.")

# Verifico los tipos de datos por columna.
print("Tipos de datos por columna después de la eliminación:")
print(df_steam_games.dtypes)

# Reviso las columnas
for col in df_steam_games.columns:
    print(f"Columna: {col}")
    print(f"Primer valor: {df_steam_games[col].iloc[0]}")

    # Convierto las listas a cadenas antes de llamar a unique()
    unique_values = pd.Series([str(x) if isinstance(x, list) else x for x in df_steam_games[col]]).unique()
    print(f"Valores únicos: {unique_values[:5]}\n")

In [ ]:
# Verifico las filas duplicadas
hay_duplicados = df_steam_games.duplicated(subset='id', keep=False).any()
print('¿Hay filas duplicadas?', hay_duplicados)

Visualizo

In [ ]:
# Columnas que no contienen listas ni diccionarios
non_list_columns = ['publisher', 'app_name', 'title', 'url', 'release_date', 'reviews_url', 'price', 'early_access', 'id', 'developer']

# Para cada columna en non_list_columns
for col in non_list_columns:
    # Imprimo el nombre de la columna
    print(f"\nAnalizando la columna '{col}'")
    print("-" * 50)
    
    # Imprimo los primeros 5 valores de la columna
    print("\nPrimeros 5 valores:")
    print(df_steam_games[col].head(5))
    
    # Imprimo los últimos 5 valores de la columna
    print("\nÚltimos 5 valores:")
    print(df_steam_games[col].tail(5))
    
    # Imprimo el número de valores únicos en la columna
    num_unique_values = df_steam_games[col].nunique()
    print(f"\nNúmero de valores únicos: {num_unique_values}")
    
    # Si el número de valores únicos es menor que 10, imprimo los valores únicos
    if num_unique_values < 10:
        print("\nValores únicos:")
        print(df_steam_games[col].unique())
    
    # Imprimo el número de valores nulos en la columna
    num_null_values = df_steam_games[col].isnull().sum()
    print(f"\nNúmero de valores nulos: {num_null_values}")


### Analizo cada columna.

Analizo Publisher

In [ ]:
# Importo librerías.
import numpy as np

def analyze_individual_column(df, column):
    """
    Esta función analiza una columna individual de un DataFrame de pandas.
    
    Parámetros:
    df (pandas.DataFrame): El DataFrame a analizar.
    column (str): El nombre de la columna a analizar.
    
    Retorna:
    None
    """
    print(f"\n\nAnalizando la columna '{column}'\n" + "-"*50)
    
    # Imprimo los primeros 20 valores
    print("\nPrimeros 20 valores:")
    print(df[column].head(20))
    
    # Verifico si hay valores nulos
    num_nulls = df[column].isnull().sum()
    print(f"\nNúmero de valores nulos: {num_nulls}")
    
    # Obtengo el tipo de datos de la columna
    dtype = df[column].dtype
    print(f"\nTipo de datos: {dtype}")
    
    # Si la columna es numérica, obtengo estadísticas descriptivas
    if pd.api.types.is_numeric_dtype(df[column]):
        print("\nEstadísticas descriptivas:")
        print(df[column].describe())
    
    # Si la columna es de tipo object (categórica), obtengo los valores únicos y sus conteos
    if dtype == 'object':
        print("\nValores únicos (top 5):")
        print(df[column].value_counts().head(5))
        
    # Si la columna contiene listas o diccionarios, analizo la longitud de los datos
    if df[column].apply(lambda x: isinstance(x, (list, dict))).any():
        print("\nLongitud de los datos (para listas y diccionarios):")
        print(df[column].apply(lambda x: len(x) if isinstance(x, (list, dict)) else None).describe())
    
    # Si la columna es de tipo int, str o datetime
    if np.issubdtype(dtype, np.integer):
        print("\nLa columna es de tipo entero.")
    elif np.issubdtype(dtype, np.datetime64):
        print("\nLa columna es de tipo datetime.")
    elif dtype == object:
        print("\nLa columna es de tipo string.")

# Uso de la función para una columna específica
df = df_steam_games  # aquí coloco el nombre del df y su nombre
column = 'publisher'  # aquí coloco el nombre de la columna

analyze_individual_column(df, column)

Verifico duplicados.

In [ ]:
# Listo nombres de publisher para verificar duplicados
app_names = ['Dovetail GAmes - Flight']

# Columnas a considerar al verificar duplicados
columns_to_consider = ['app_name', 'publisher', 'developer']

# Para cada nombre de juego en la lista
for game_name in app_names:  # Aquí está la corrección
    # Crea una cadena de texto con la siguiente información
    info = f"""
    Este bloque de código verifica si hay filas duplicadas para el juego 
    '{game_name}' en el DataFrame df_steam_games. 
    Si hay filas duplicadas, las imprime y luego las elimina del DataFrame.
    """
    # Imprimo la cadena de texto
    print(info)

    # Filtro el DataFrame para mostrar solo las filas donde 'app_name' es el nombre del juego actual
    df_game = df_steam_games[df_steam_games['app_name'] == game_name]

    # Verifico si hay filas duplicadas, considerando solo las columnas especificadas
    duplicates = df_game.duplicated(subset=columns_to_consider)

    # Si hay filas duplicadas
    if duplicates.any():
        print(f"Duplicados para {game_name}:")
        
        # Imprimo las filas duplicadas
        print(df_game[duplicates])
    else:
        # Imprimo un mensaje indicando que no se encontraron duplicados para el juego actual
        print(f"No se encontraron duplicados para {game_name}.")


In [ ]:
# Analizo 'genres', 'tags' y 'specs'

# Importo librerías.
from tqdm import tqdm

def analizar_columna_individual(df, columna):
    """
    Esta función analiza cada columna individualmente, en este caso 'genres', 'tags' y 'specs' de un DataFrame de pandas.
    
    Parámetros:
    df (pandas.DataFrame): El DataFrame a analizar.
    columna (str): El nombre de la columna a analizar.
    
    Retorna:
    None
    """
    # Verifico si la columna existe en el DataFrame
    if columna not in df.columns:
        print(f"La columna '{columna}' no existe en el DataFrame.")
        return

    # Verifico si hay valores nulos
    num_nulls = df[columna].isnull().sum()
    
    # Obtengo un resumen de la columna
    resumen = df[columna].describe()

    print(f"Columna '{columna}':\n{resumen}\nNúmero de valores nulos = {num_nulls}\n")

# Uso de la función para cada columna nueva
for columna in tqdm(new_column_genres + new_colum_tags + new_colum_specs):
    analizar_columna_individual(df_steam_games, columna)

Verifico duplicados

Analizo app_name

In [ ]:
# Importo librerías.
import pandas as pd
import numpy as np

def analyze_individual_column(df, column):
    """
    Esta función analiza una columna individual de un DataFrame de pandas.
    
    Parámetros:
    df (pandas.DataFrame): El DataFrame a analizar.
    column (str): El nombre de la columna a analizar.
    
    Retorna:
    None
    """
    print(f"\n\nAnalizando la columna '{column}'\n" + "-"*50)
    
    # Imprimo los primeros 20 valores
    print("\nPrimeros 20 valores:")
    print(df[column].head(20))
    
    # Verifico si hay valores nulos
    num_nulls = df[column].isnull().sum()
    print(f"\nNúmero de valores nulos: {num_nulls}")
    
    # Obtengo el tipo de datos de la columna
    dtype = df[column].dtype
    print(f"\nTipo de datos: {dtype}")
    
    # Si la columna es numérica, obtengo estadísticas descriptivas
    if pd.api.types.is_numeric_dtype(df[column]):
        print("\nEstadísticas descriptivas:")
        print(df[column].describe())
    
    # Si la columna es de tipo object (categórica), obtengo los valores únicos y sus conteos
    if dtype == 'object':
        print("\nValores únicos (top 5):")
        print(df[column].value_counts().head(5))
        
    # Si la columna contiene listas o diccionarios, analizo la longitud de los datos
    if df[column].apply(lambda x: isinstance(x, (list, dict))).any():
        print("\nLongitud de los datos (para listas y diccionarios):")
        print(df[column].apply(lambda x: len(x) if isinstance(x, (list, dict)) else None).describe())
    
    # Si la columna es de tipo int, str o datetime
    if np.issubdtype(dtype, np.integer):
        print("\nLa columna es de tipo entero.")
    elif np.issubdtype(dtype, np.datetime64):
        print("\nLa columna es de tipo datetime.")
    elif dtype == object:
        print("\nLa columna es de tipo string.")

# Uso de la función para una columna específica
df = df_steam_games  # aquí coloco el nombre del df y su nombre
column = 'app_name'  # aquí coloco el nombre de la columna

analyze_individual_column(df, column)


Verifico duplicados

In [ ]:
# Listo nombres de app_names para verificar duplicados
app_names = ['Invisible Apartment Zero']

# Columnas a considerar al verificar duplicados
columns_to_consider = ['app_name', 'publisher', 'developer', 'title']

# Para cada nombre de juego en la lista
for game_name in app_names:  # Aquí está la corrección
    # Creo una cadena de texto con la siguiente información
    info = f"""
    Este bloque de código verifica si hay filas duplicadas para el juego 
    '{game_name}' en el DataFrame df_steam_games. 
    Si hay filas duplicadas, las imprime y luego las elimina del DataFrame.
    """
    # Imprimo la cadena de texto
    print(info)

    # Filtro el DataFrame para mostrar solo las filas donde 'app_name' es el nombre del juego actual
    df_game = df_steam_games[df_steam_games['app_name'] == game_name]

    # Verifico si hay filas duplicadas, considerando solo las columnas especificadas
    duplicates = df_game.duplicated(subset=columns_to_consider)

    # Si hay filas duplicadas
    if duplicates.any():
        print(f"Duplicados para {game_name}:")
        
        # Imprimo las filas duplicadas
        print(df_game[duplicates])
    else:
        # Imprimo un mensaje indicando que no se encontraron duplicados para el juego actual
        print(f"No se encontraron duplicados para {game_name}.")


Analizo title

In [ ]:
# Importo librerías
import pandas as pd
import numpy as np

def analyze_individual_column(df, column):
    """
    Esta función analiza una columna individual de un DataFrame de pandas.
    
    Parámetros:
    df (pandas.DataFrame): El DataFrame a analizar.
    column (str): El nombre de la columna a analizar.
    
    Retorna:
    None
    """
    print(f"\n\nAnalizando la columna '{column}'\n" + "-"*50)
    
    # Imprimo los primeros 20 valores
    print("\nPrimeros 20 valores:")
    print(df[column].head(20))
    
    # Verifico si hay valores nulos
    num_nulls = df[column].isnull().sum()
    print(f"\nNúmero de valores nulos: {num_nulls}")
    
    # Obtengo el tipo de datos de la columna
    dtype = df[column].dtype
    print(f"\nTipo de datos: {dtype}")
    
    # Si la columna es numérica, obtengo estadísticas descriptivas
    if pd.api.types.is_numeric_dtype(df[column]):
        print("\nEstadísticas descriptivas:")
        print(df[column].describe())
    
    # Si la columna es de tipo object (categórica), obtengo los valores únicos y sus conteos
    if dtype == 'object':
        print("\nValores únicos (top 5):")
        print(df[column].value_counts().head(5))
        
    # Si la columna contiene listas o diccionarios, analizo la longitud de los datos
    if df[column].apply(lambda x: isinstance(x, (list, dict))).any():
        print("\nLongitud de los datos (para listas y diccionarios):")
        print(df[column].apply(lambda x: len(x) if isinstance(x, (list, dict)) else None).describe())
    
    # Si la columna es de tipo int, str o datetime
    if np.issubdtype(dtype, np.integer):
        print("\nLa columna es de tipo entero.")
    elif np.issubdtype(dtype, np.datetime64):
        print("\nLa columna es de tipo datetime.")
    elif dtype == object:
        print("\nLa columna es de tipo string.")

# Uso de la función para una columna específica
df = df_steam_games  # aquí coloco el nombre del df y su nombre
column = 'title'  # aquí coloco el nombre de la columna

analyze_individual_column(df, column)


Verifico duplicados

In [ ]:
# Listo nombres de title para verificar duplicados
app_names = ['Invisible Apartment Zero']

# Columnas a considerar al verificar duplicados
columns_to_consider = ['app_name', 'publisher', 'developer', 'id']

# Para cada nombre de juego en la lista
for game_name in app_names:  # Aquí se debe usar app_names en lugar de game_name
    # Creo una cadena de texto con la siguiente información
    info = f"""
    Este bloque de código verifica si hay filas duplicadas para el juego 
    '{game_name}' en el DataFrame df_steam_games. 
    Si hay filas duplicadas, las imprime.
    """
    # Imprimo la cadena de texto
    print(info)

    # Filtro el DataFrame para mostrar solo las filas donde 'app_name' es el nombre del juego actual
    df_game = df_steam_games[df_steam_games['app_name'] == game_name]

    # Verifico si hay filas duplicadas, considerando solo las columnas especificadas
    duplicates = df_game.duplicated(subset=columns_to_consider)

    # Si hay filas duplicadas
    if duplicates.any():
        print(f"Duplicados para {game_name}:")
        
        # Imprimo las filas duplicadas
        print(df_game[duplicates])
    else:
        # Imprimo un mensaje indicando que no se encontraron duplicados para el juego actual
        print(f"No se encontraron duplicados para {game_name}.") 


Analizo url

In [ ]:
# Importo librerías
import pandas as pd
import numpy as np

def analyze_individual_column(df, column):
    """
    Esta función analiza una columna individual de un DataFrame de pandas.
    
    Parámetros:
    df (pandas.DataFrame): El DataFrame a analizar.
    column (str): El nombre de la columna a analizar.
    
    Retorna:
    None
    """
    print(f"\n\nAnalizando la columna '{column}'\n" + "-"*50)
    
    # Imprimo los primeros 20 valores
    print("\nPrimeros 20 valores:")
    print(df[column].head(20))
    
    # Verifico si hay valores nulos
    num_nulls = df[column].isnull().sum()
    print(f"\nNúmero de valores nulos: {num_nulls}")
    
    # Obtengo el tipo de datos de la columna
    dtype = df[column].dtype
    print(f"\nTipo de datos: {dtype}")
    
    # Si la columna es numérica, obtengo estadísticas descriptivas
    if pd.api.types.is_numeric_dtype(df[column]):
        print("\nEstadísticas descriptivas:")
        print(df[column].describe())
    
    # Si la columna es de tipo object (categórica), obtengo los valores únicos y sus conteos
    if dtype == 'object':
        print("\nValores únicos (top 5):")
        print(df[column].value_counts().head(5))
        
    # Si la columna contiene listas o diccionarios, analizo la longitud de los datos
    if df[column].apply(lambda x: isinstance(x, (list, dict))).any():
        print("\nLongitud de los datos (para listas y diccionarios):")
        print(df[column].apply(lambda x: len(x) if isinstance(x, (list, dict)) else None).describe())
    
    # Si la columna es de tipo int, str o datetime
    if np.issubdtype(dtype, np.integer):
        print("\nLa columna es de tipo entero.")
    elif np.issubdtype(dtype, np.datetime64):
        print("\nLa columna es de tipo datetime.")
    elif dtype == object:
        print("\nLa columna es de tipo string.")

# Uso de la función para una columna específica
df = df_steam_games  # aquí coloco el nombre del df y su nombre
column = 'url'  # aquí coloco el nombre de la columna

analyze_individual_column(df, column)


Verifico duplicados

In [ ]:
# Listo los links para verificar duplicados
app_links = ['http://store.steampowered.com/app/454700/Shiny_Gauntlet/']

# Columnas a considerar al verificar duplicados
columns_to_consider = ['app_name', 'publisher', 'developer']

# Para cada link de juego en la lista
for game_link in app_links:  # Aquí se debe usar app_links en lugar de game_names
    # Crea una cadena de texto con la información que deseas
    info = f"""
    Este bloque de código verifica si hay filas duplicadas para el juego 
    '{game_link}' en el DataFrame df_steam_games. 
    Si hay filas duplicadas, las imprime.
    """
    # Imprimo la cadena de texto
    print(info)

    # Filtro el DataFrame para mostrar solo las filas donde 'app_links' es el link del juego actual
    df_game = df_steam_games[df_steam_games['app_name'] == game_link]  # Aquí se debe usar game_link en lugar de game_name

    # Verifico si hay filas duplicadas, considerando solo las columnas especificadas
    duplicates = df_game.duplicated(subset=columns_to_consider)

    # Si hay filas duplicadas
    if duplicates.any():
        print(f"Duplicados para {game_link}:")  # Aquí se debe usar game_link en lugar de game_name
        
        # Imprime las filas duplicadas
        print(df_game[duplicates])
    else:
        # Imprime un mensaje indicando que no se encontraron duplicados para el juego actual
        print(f"No se encontraron duplicados para {game_link}.")  # Aquí se debe usar game_link en lugar de game_name


Analizo release_date

In [ ]:
# Importo librerías
import pandas as pd
import numpy as np

def analyze_individual_column(df, column):
    """
    Esta función analiza una columna individual de un DataFrame de pandas.
    
    Parámetros:
    df (pandas.DataFrame): El DataFrame a analizar.
    column (str): El nombre de la columna a analizar.
    
    Retorna:
    None
    """
    print(f"\n\nAnalizando la columna '{column}'\n" + "-"*50)
    
    # Imprimo los primeros 20 valores
    print("\nPrimeros 20 valores:")
    print(df[column].head(20))
    
    # Verifico si hay valores nulos
    num_nulls = df[column].isnull().sum()
    print(f"\nNúmero de valores nulos: {num_nulls}")
    
    # Obtengo el tipo de datos de la columna
    dtype = df[column].dtype
    print(f"\nTipo de datos: {dtype}")
    
    # Si la columna es numérica, obtengo estadísticas descriptivas
    if pd.api.types.is_numeric_dtype(df[column]):
        print("\nEstadísticas descriptivas:")
        print(df[column].describe())
    
    # Si la columna es de tipo object (categórica), obtengo los valores únicos y sus conteos
    if dtype == 'object':
        print("\nValores únicos (top 5):")
        print(df[column].value_counts().head(5))
        
    # Si la columna contiene listas o diccionarios, analizo la longitud de los datos
    if df[column].apply(lambda x: isinstance(x, (list, dict))).any():
        print("\nLongitud de los datos (para listas y diccionarios):")
        print(df[column].apply(lambda x: len(x) if isinstance(x, (list, dict)) else None).describe())
    
    # Si la columna es de tipo int, str o datetime
    if np.issubdtype(dtype, np.integer):
        print("\nLa columna es de tipo entero.")
    elif np.issubdtype(dtype, np.datetime64):
        print("\nLa columna es de tipo datetime.")
    elif dtype == object:
        print("\nLa columna es de tipo string.")

# Uso de la función para una columna específica
df = df_steam_games  # aquí coloco el nombre del df y su nombre
column = 'release_date'  # aquí coloco el nombre de la columna

analyze_individual_column(df, column)


Verifico duplicados.

In [ ]:
# Listo las fechas para verificar duplicados
release_dates = ['2017-12-15']

# Columnas a considerar al verificar duplicados
columns_to_consider = ['release_date', 'publisher', 'developer', 'title']

# Para cada fecha de lanzamiento en la lista
for release_date in release_dates:  
    # Crea una cadena de texto con la siguiente información:
    info = f"""
    Este bloque de código verifica si hay filas duplicadas para la fecha de lanzamiento 
    '{release_date}' en el DataFrame df_steam_games. 
    Si hay filas duplicadas, las imprime.
    """
    # Imprimo la cadena de texto
    print(info)

    # Filtro el DataFrame para mostrar solo las filas donde 'release_date' es la fecha de lanzamiento actual
    df_date = df_steam_games[df_steam_games['release_date'] == release_date]

    # Verifico si hay filas duplicadas, considerando solo las columnas especificadas
    duplicates = df_date.duplicated(subset=columns_to_consider)

    # Si hay filas duplicadas
    if duplicates.any():
        print(f"Duplicados para {release_date}:")  
        
        # Imprimo las filas duplicadas
        print(df_date[duplicates])
    else:
        # Imprimo un mensaje indicando que no se encontraron duplicados para la fecha de lanzamiento actual
        print(f"No se encontraron duplicados para {release_date}.")


Cambiamos el contenido de release_date a datetime

In [ ]:
# Convierto la columna 'release_date' a datetime
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')

# Ahora puedes volver a llamar a la función analyze_individual_column()
analyze_individual_column(df, 'release_date')


Analizo reviews_url

In [ ]:
# Importo librerías
import pandas as pd
import numpy as np

def analyze_individual_column(df, column):
    """
    Esta función analiza una columna individual de un DataFrame de pandas.
    
    Parámetros:
    df (pandas.DataFrame): El DataFrame a analizar.
    column (str): El nombre de la columna a analizar.
    
    Retorna:
    None
    """
    print(f"\n\nAnalizando la columna '{column}'\n" + "-"*50)
    
    # Imprimo los primeros 20 valores
    print("\nPrimeros 20 valores:")
    print(df[column].head(20))
    
    # Verifico si hay valores nulos
    num_nulls = df[column].isnull().sum()
    print(f"\nNúmero de valores nulos: {num_nulls}")
    
    # Obtengo el tipo de datos de la columna
    dtype = df[column].dtype
    print(f"\nTipo de datos: {dtype}")
    
    # Si la columna es numérica, obtengo estadísticas descriptivas
    if pd.api.types.is_numeric_dtype(df[column]):
        print("\nEstadísticas descriptivas:")
        print(df[column].describe())
    
    # Si la columna es de tipo object (categórica), obtengo los valores únicos y sus conteos
    if dtype == 'object':
        print("\nValores únicos (top 5):")
        print(df[column].value_counts().head(5))
        
    # Si la columna contiene listas o diccionarios, analizo la longitud de los datos
    if df[column].apply(lambda x: isinstance(x, (list, dict))).any():
        print("\nLongitud de los datos (para listas y diccionarios):")
        print(df[column].apply(lambda x: len(x) if isinstance(x, (list, dict)) else None).describe())
    
    # Si la columna es de tipo int, str o datetime
    if np.issubdtype(dtype, np.integer):
        print("\nLa columna es de tipo entero.")
    elif np.issubdtype(dtype, np.datetime64):
        print("\nLa columna es de tipo datetime.")
    elif dtype == object:
        print("\nLa columna es de tipo string.")

# Uso de la función para una columna específica
df = df_steam_games  # aquí coloco el nombre del df y su nombre
column = 'reviews_url'  # aquí coloco el nombre de la columna

analyze_individual_column(df, column)


Verifico duplicados

In [ ]:
# Listo las URLs para verificar duplicados
review_urls = ['http://steamcommunity.com/app/454700/reviews/?browsefilter=mostrecent&p=1']

# Columnas a considerar al verificar duplicados
columns_to_consider = ['reviews_url', 'publisher', 'developer', 'title', 'url']

# Para cada URL de revisión en la lista
for review_url in review_urls:  
    # Crea una cadena de texto con la siguiente información:
    info = f"""
    Este bloque de código verifica si hay filas duplicadas para la URL de revisión 
    '{review_url}' en el DataFrame df_steam_games. 
    Si hay filas duplicadas, las imprime.
    """
    # Imprimo la cadena de texto
    print(info)

    # Filtro el DataFrame para mostrar solo las filas donde 'reviews_url' es la URL de revisión actual
    df_reviews = df_steam_games[df_steam_games['reviews_url'] == review_url]

    # Verifico si hay filas duplicadas, considerando solo las columnas especificadas
    duplicates = df_reviews.duplicated(subset=columns_to_consider)

    # Si hay filas duplicadas
    if duplicates.any():
        print(f"Duplicados para {review_url}:")  
        
        # Imprimo las filas duplicadas
        print(df_reviews[duplicates])
    else:
        # Imprimo un mensaje indicando que no se encontraron duplicados para la URL de revisión actual
        print(f"No se encontraron duplicados para {review_url}.")


Analizo Price

In [ ]:
# Importo librerías
import pandas as pd
import numpy as np

def analyze_individual_column(df, column):
    """
    Esta función analiza una columna individual de un DataFrame de pandas.
    
    Parámetros:
    df (pandas.DataFrame): El DataFrame a analizar.
    column (str): El nombre de la columna a analizar.
    
    Retorna:
    None
    """
    print(f"\n\nAnalizando la columna '{column}'\n" + "-"*50)
    
    # Imprimo los primeros 20 valores
    print("\nPrimeros 20 valores:")
    print(df[column].head(20))
    
    # Verifico si hay valores nulos
    num_nulls = df[column].isnull().sum()
    print(f"\nNúmero de valores nulos: {num_nulls}")
    
    # Obtengo el tipo de datos de la columna
    dtype = df[column].dtype
    print(f"\nTipo de datos: {dtype}")
    
    # Si la columna es numérica, obtengo estadísticas descriptivas
    if pd.api.types.is_numeric_dtype(df[column]):
        print("\nEstadísticas descriptivas:")
        print(df[column].describe())
    
    # Si la columna es de tipo object (categórica), obtengo los valores únicos y sus conteos
    if dtype == 'object':
        print("\nValores únicos (top 5):")
        print(df[column].value_counts().head(5))
        
    # Si la columna contiene listas o diccionarios, analizo la longitud de los datos
    if df[column].apply(lambda x: isinstance(x, (list, dict))).any():
        print("\nLongitud de los datos (para listas y diccionarios):")
        print(df[column].apply(lambda x: len(x) if isinstance(x, (list, dict)) else None).describe())
    
    # Si la columna es de tipo int, str o datetime
    if np.issubdtype(dtype, np.integer):
        print("\nLa columna es de tipo entero.")
    elif np.issubdtype(dtype, np.datetime64):
        print("\nLa columna es de tipo datetime.")
    elif dtype == object:
        print("\nLa columna es de tipo string.")

# Uso de la función para una columna específica
df = df_steam_games  # aquí coloco el nombre del df y su nombre
column = 'price'  # aquí coloco el nombre de la columna

analyze_individual_column(df, column)


Verifico duplicados

In [ ]:
# Listo los precios para verificar duplicados
prices = [2.99]

# Columnas a considerar al verificar duplicados
columns_to_consider = ['release_date', 'publisher', 'developer', 'title']

# Para cada precio en la lista
for price in prices:  
    # Crea una cadena de texto con la siguiente información:
    info = f"""
    Este bloque de código verifica si hay filas duplicadas para el precio 
    '{price}' en el DataFrame df_steam_games. 
    Si hay filas duplicadas, las imprime.
    """
    # Imprimo la cadena de texto
    print(info)

    # Filtro el DataFrame para mostrar solo las filas donde 'price' es el precio actual
    df_price = df_steam_games[df_steam_games['price'] == price]

    # Verifico si hay filas duplicadas, considerando solo las columnas especificadas
    duplicates = df_price.duplicated(subset=columns_to_consider)

    # Si hay filas duplicadas
    if duplicates.any():
        print(f"Duplicados para {price}:")  
        
        # Imprimo las filas duplicadas
        print(df_price[duplicates])
    else:
        # Imprimo un mensaje indicando que no se encontraron duplicados para el precio actual
        print(f"No se encontraron duplicados para {price}.")


In [ ]:
# Cuento cuántos datos en la columna 'price' son strings
num_strings = df['price'].apply(lambda x: isinstance(x, str)).sum()

# Calculo el porcentaje de datos que son strings
percentage_strings = (num_strings / len(df)) * 100

print(f"Porcentaje de strings en 'price': {percentage_strings}%")

Analizo early_access

In [ ]:
# Importo librerías
import pandas as pd
import numpy as np

def analyze_individual_column(df, column):
    """
    Esta función analiza una columna individual de un DataFrame de pandas.
    
    Parámetros:
    df (pandas.DataFrame): El DataFrame a analizar.
    column (str): El nombre de la columna a analizar.
    
    Retorna:
    None
    """
    print(f"\n\nAnalizando la columna '{column}'\n" + "-"*50)
    
    # Imprimo los primeros 20 valores
    print("\nPrimeros 20 valores:")
    print(df[column].head(20))
    
    # Verifico si hay valores nulos
    num_nulls = df[column].isnull().sum()
    print(f"\nNúmero de valores nulos: {num_nulls}")
    
    # Obtengo el tipo de datos de la columna
    dtype = df[column].dtype
    print(f"\nTipo de datos: {dtype}")
    
    # Si la columna es numérica, obtengo estadísticas descriptivas
    if pd.api.types.is_numeric_dtype(df[column]):
        print("\nEstadísticas descriptivas:")
        print(df[column].describe())
    
    # Si la columna es de tipo object (categórica), obtengo los valores únicos y sus conteos
    if dtype == 'object':
        print("\nValores únicos (top 5):")
        print(df[column].value_counts().head(5))
        
    # Si la columna contiene listas o diccionarios, analizo la longitud de los datos
    if df[column].apply(lambda x: isinstance(x, (list, dict))).any():
        print("\nLongitud de los datos (para listas y diccionarios):")
        print(df[column].apply(lambda x: len(x) if isinstance(x, (list, dict)) else None).describe())
    
    # Si la columna es de tipo int, str o datetime
    if np.issubdtype(dtype, np.integer):
        print("\nLa columna es de tipo entero.")
    elif np.issubdtype(dtype, np.datetime64):
        print("\nLa columna es de tipo datetime.")
    elif dtype == object:
        print("\nLa columna es de tipo string.")

# Uso de la función para una columna específica
df = df_steam_games  # aquí coloco el nombre del df y su nombre
column = 'early_access'  # aquí coloco el nombre de la columna

analyze_individual_column(df, column)


Verifico duplicados.

In [ ]:
# Listo los valores para verificar duplicados
early_access_values = [True]

# Columnas a considerar al verificar duplicados
columns_to_consider = ['early_access', 'publisher', 'developer', 'title', 'price', 'id', 'app_name']

# Para cada valor en la lista
for early_access in early_access_values:  
    # Crea una cadena de texto con la siguiente información:
    info = f"""
    Este bloque de código verifica si hay filas duplicadas para el valor de 'early_access' 
    '{early_access}' en el DataFrame df_steam_games. 
    Si hay filas duplicadas, las imprime.
    """
    # Imprimo la cadena de texto
    print(info)

    # Filtro el DataFrame para mostrar solo las filas donde 'early_access' es el valor actual
    df_early_access = df_steam_games[df_steam_games['early_access'] == early_access]

    # Verifico si hay filas duplicadas, considerando solo las columnas especificadas
    duplicates = df_early_access.duplicated(subset=columns_to_consider)

    # Si hay filas duplicadas
    if duplicates.any():
        print(f"Duplicados para {early_access}:")  
        
        # Imprimo las filas duplicadas
        print(df_early_access[duplicates])
    else:
        # Imprimo un mensaje indicando que no se encontraron duplicados para el valor de 'early_access' actual
        print(f"No se encontraron duplicados para {early_access}.")


Analizo id

In [ ]:
# Importo librerías
import pandas as pd
import numpy as np

def analyze_individual_column(df, column):
    """
    Esta función analiza una columna individual de un DataFrame de pandas.
    
    Parámetros:
    df (pandas.DataFrame): El DataFrame a analizar.
    column (str): El nombre de la columna a analizar.
    
    Retorna:
    None
    """
    print(f"\n\nAnalizando la columna '{column}'\n" + "-"*50)
    
    # Imprimo los primeros 20 valores
    print("\nPrimeros 20 valores:")
    print(df[column].head(20))
    
    # Verifico si hay valores nulos
    num_nulls = df[column].isnull().sum()
    print(f"\nNúmero de valores nulos: {num_nulls}")
    
    # Obtengo el tipo de datos de la columna
    dtype = df[column].dtype
    print(f"\nTipo de datos: {dtype}")
    
    # Si la columna es numérica, obtengo estadísticas descriptivas
    if pd.api.types.is_numeric_dtype(df[column]):
        print("\nEstadísticas descriptivas:")
        print(df[column].describe())
    
    # Si la columna es de tipo object (categórica), obtengo los valores únicos y sus conteos
    if dtype == 'object':
        print("\nValores únicos (top 5):")
        print(df[column].value_counts().head(5))
        
    # Si la columna contiene listas o diccionarios, analizo la longitud de los datos
    if df[column].apply(lambda x: isinstance(x, (list, dict))).any():
        print("\nLongitud de los datos (para listas y diccionarios):")
        print(df[column].apply(lambda x: len(x) if isinstance(x, (list, dict)) else None).describe())
    
    # Si la columna es de tipo int, str o datetime
    if np.issubdtype(dtype, np.integer):
        print("\nLa columna es de tipo entero.")
    elif np.issubdtype(dtype, np.datetime64):
        print("\nLa columna es de tipo datetime.")
    elif dtype == object:
        print("\nLa columna es de tipo string.")

# Uso de la función para una columna específica
df = df_steam_games  # aquí coloco el nombre del df y su nombre
column = 'id'  # aquí coloco el nombre de la columna

analyze_individual_column(df, column)


Verifico duplicados.

In [ ]:
# Listo los IDs para verificar duplicados
ids = [454700]

# Columnas a considerar al verificar duplicados
columns_to_consider = ['id', 'publisher', 'developer', 'title', 'price', 'early_access']

# Para cada ID en la lista
for id in ids:  
    # Crea una cadena de texto con la siguiente información:
    info = f"""
    Este bloque de código verifica si hay filas duplicadas para el ID 
    '{id}' en el DataFrame df_steam_games. 
    Si hay filas duplicadas, las imprime.
    """
    # Imprimo la cadena de texto
    print(info)

    # Filtro el DataFrame para mostrar solo las filas donde 'id' es el ID actual
    df_id = df_steam_games[df_steam_games['id'] == id]

    # Verifico si hay filas duplicadas, considerando solo las columnas especificadas
    duplicates = df_id.duplicated(subset=columns_to_consider)

    # Si hay filas duplicadas
    if duplicates.any():
        print(f"Duplicados para {id}:")  
        
        # Imprimo las filas duplicadas
        print(df_id[duplicates])
    else:
        # Imprimo un mensaje indicando que no se encontraron duplicados para el ID actual
        print(f"No se encontraron duplicados para {id}.")


Modifico id a int

In [ ]:
# Intento convertir la columna 'id' a entero
try:
    df['id'] = df['id'].astype(int)
    print("La columna 'id' se ha convertido a entero exitosamente.")
except Exception as e:
    print(f"Error al convertir la columna 'id' a entero: {e}")

# Ahora puedes volver a llamar a la función analyze_individual_column()
analyze_individual_column(df, 'id')


In [ ]:
# Verifico
num_nan = df['id'].isna().sum()
print(f"Número de NaN en 'id': {num_nan}")


Analizo developer

In [ ]:
# Importo librerías
import pandas as pd
import numpy as np

def analyze_individual_column(df, column):
    """
    Esta función analiza una columna individual de un DataFrame de pandas.
    
    Parámetros:
    df (pandas.DataFrame): El DataFrame a analizar.
    column (str): El nombre de la columna a analizar.
    
    Retorna:
    None
    """
    print(f"\n\nAnalizando la columna '{column}'\n" + "-"*50)
    
    # Imprimo los primeros 20 valores
    print("\nPrimeros 20 valores:")
    print(df[column].head(20))
    
    # Verifico si hay valores nulos
    num_nulls = df[column].isnull().sum()
    print(f"\nNúmero de valores nulos: {num_nulls}")
    
    # Obtengo el tipo de datos de la columna
    dtype = df[column].dtype
    print(f"\nTipo de datos: {dtype}")
    
    # Si la columna es numérica, obtengo estadísticas descriptivas
    if pd.api.types.is_numeric_dtype(df[column]):
        print("\nEstadísticas descriptivas:")
        print(df[column].describe())
    
    # Si la columna es de tipo object (categórica), obtengo los valores únicos y sus conteos
    if dtype == 'object':
        print("\nValores únicos (top 5):")
        print(df[column].value_counts().head(5))
        
    # Si la columna contiene listas o diccionarios, analizo la longitud de los datos
    if df[column].apply(lambda x: isinstance(x, (list, dict))).any():
        print("\nLongitud de los datos (para listas y diccionarios):")
        print(df[column].apply(lambda x: len(x) if isinstance(x, (list, dict)) else None).describe())
    
    # Si la columna es de tipo int, str o datetime
    if np.issubdtype(dtype, np.integer):
        print("\nLa columna es de tipo entero.")
    elif np.issubdtype(dtype, np.datetime64):
        print("\nLa columna es de tipo datetime.")
    elif dtype == object:
        print("\nLa columna es de tipo string.")

# Uso de la función para una columna específica
df = df_steam_games  # aquí coloco el nombre del df y su nombre
column = 'developer'  # aquí coloco el nombre de la columna

analyze_individual_column(df, column)


Verifico duplicados.

In [ ]:
# Listo los desarrolladores para verificar duplicados
developers = ['Paradox Development Studio']

# Columnas a considerar al verificar duplicados
columns_to_consider = ['developer', 'publisher', 'title', 'price', 'early_access', 'id']

# Para cada desarrollador en la lista
for developer in developers:  
    # Crea una cadena de texto con la siguiente información:
    info = f"""
    Este bloque de código verifica si hay filas duplicadas para el desarrollador 
    '{developer}' en el DataFrame df_steam_games. 
    Si hay filas duplicadas, las imprime.
    """
    # Imprimo la cadena de texto
    print(info)

    # Filtro el DataFrame para mostrar solo las filas donde 'developer' es el desarrollador actual
    df_developer = df_steam_games[df_steam_games['developer'] == developer]

    # Verifico si hay filas duplicadas, considerando solo las columnas especificadas
    duplicates = df_developer.duplicated(subset=columns_to_consider)

    # Si hay filas duplicadas
    if duplicates.any():
        print(f"Duplicados para {developer}:")  
        
        # Imprimo las filas duplicadas
        print(df_developer[duplicates])
    else:
        # Imprimo un mensaje indicando que no se encontraron duplicados para el desarrollador actual
        print(f"No se encontraron duplicados para {developer}.")



<h5 style="color:blue">
Trabajaré con user reviews.

Verifico.

In [2]:
df_user_reviews


,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


Verifico las columnas que contienen listas o diccionarios.

In [3]:
# Importo librerías.
import pandas as pd

# Itero sobre cada columna
for column in df_user_reviews.columns:
    # Obtengo el primer valor no nulo
    non_null_values = df_user_reviews[column].dropna()
    if non_null_values.empty:
        print(f"La columna '{column}' solo contiene valores nulos.")
        continue
    first_non_null_value = non_null_values.iloc[0]

    # Compruebo si es una lista o un diccionario
    if isinstance(first_non_null_value, list):
        print(f"La columna '{column}' contiene una lista.")
    elif isinstance(first_non_null_value, dict):
        print(f"La columna '{column}' contiene un diccionario.")
    else:
        print(f"La columna '{column}' no contiene ni listas ni diccionarios.")


La columna 'user_id' no contiene ni listas ni diccionarios.
La columna 'user_url' no contiene ni listas ni diccionarios.
La columna 'reviews' contiene una lista.


Elimino duplicados

In [ ]:
# Convierto los diccionarios en 'reviews' a strings
df_user_reviews['reviews'] = df_user_reviews['reviews'].apply(str)

# Ahora sí puedo eliminar duplicados
df_user_reviews.drop_duplicates(subset=['user_id', 'user_url', 'reviews'], inplace=True)


Analizo user id

In [ ]:
# Importo librerías
import pandas as pd
import numpy as np

def analyze_individual_column(df, column):
    """
    Esta función analiza una columna individual de un DataFrame de pandas.
    
    Parámetros:
    df (pandas.DataFrame): El DataFrame a analizar.
    column (str): El nombre de la columna a analizar.
    
    Retorna:
    None
    """
    print(f"\n\nAnalizando la columna '{column}'\n" + "-"*50)
    
    # Imprimo los primeros 20 valores
    print("\nPrimeros 20 valores:")
    print(df[column].head(20))
    
    # Verifico si hay valores nulos
    num_nulls = df[column].isnull().sum()
    print(f"\nNúmero de valores nulos: {num_nulls}")
    
    # Obtengo el tipo de datos de la columna
    dtype = df[column].dtype
    print(f"\nTipo de datos: {dtype}")
    
    # Si la columna es numérica, obtengo estadísticas descriptivas
    if pd.api.types.is_numeric_dtype(df[column]):
        print("\nEstadísticas descriptivas:")
        print(df[column].describe())
    
    # Si la columna es de tipo object (categórica), obtengo los valores únicos y sus conteos
    if dtype == 'object':
        print("\nValores únicos (top 5):")
        print(df[column].value_counts().head(5))
        
    # Si la columna contiene listas o diccionarios, analizo la longitud de los datos
    if df[column].apply(lambda x: isinstance(x, (list, dict))).any():
        print("\nLongitud de los datos (para listas y diccionarios):")
        print(df[column].apply(lambda x: len(x) if isinstance(x, (list, dict)) else None).describe())
    
    # Si la columna es de tipo int, str o datetime
    if np.issubdtype(dtype, np.integer):
        print("\nLa columna es de tipo entero.")
    elif np.issubdtype(dtype, np.datetime64):
        print("\nLa columna es de tipo datetime.")
    elif dtype == object:
        print("\nLa columna es de tipo string.")

# Uso de la función para una columna específica
df = df_user_reviews  # aquí coloco el nombre del df y su nombre
column = 'user_id'  # aquí coloco el nombre de la columna

analyze_individual_column(df, column)


Verifico duplicados.

In [ ]:
# Listo los IDs de usuario para verificar duplicados
user_ids = ['76561198061820192']  # Aquí puedes poner los IDs de usuario que quieras verificar

# Columnas a considerar al verificar duplicados
# Aquí puedes poner las columnas que quieras considerar al verificar duplicados
columns_to_consider = ['user_id', 'user_url', 'reviews']

# Para cada ID de usuario en la lista
for user_id in user_ids:  
    # Crea una cadena de texto con la siguiente información:
    info = f"""
    Este bloque de código verifica si hay filas duplicadas para el ID de usuario 
    '{user_id}' en el DataFrame df_user_reviews. 
    Si hay filas duplicadas, las imprime.
    """
    # Imprimo la cadena de texto
    print(info)

    # Filtro el DataFrame para mostrar solo las filas donde 'user_id' es el ID de usuario actual
    df_id = df_user_reviews[df_user_reviews['user_id'] == user_id].copy()

    # Convierto las listas en la columna 'reviews' a una representación de cadena
    df_id['reviews'] = df_id['reviews'].apply(str)

    # Verifico si hay filas duplicadas, considerando solo las columnas especificadas
    duplicates = df_id.duplicated(subset=columns_to_consider)

    # Si hay filas duplicadas
    if duplicates.any():
        print(f"Duplicados para {user_id}:")  
        
        # Imprimo las filas duplicadas
        print(df_id[duplicates])
    else:
        # Imprimo un mensaje indicando que no se encontraron duplicados para el ID de usuario actual
        print(f"No se encontraron duplicados para {user_id}.")


Analizo user url

In [ ]:
# Importo librerías
import pandas as pd
import numpy as np

def analyze_individual_column(df, column):
    """
    Esta función analiza una columna individual de un DataFrame de pandas.
    
    Parámetros:
    df (pandas.DataFrame): El DataFrame a analizar.
    column (str): El nombre de la columna a analizar.
    
    Retorna:
    None
    """
    print(f"\n\nAnalizando la columna '{column}'\n" + "-"*50)
    
    # Imprimo los primeros 20 valores
    print("\nPrimeros 20 valores:")
    print(df[column].head(20))
    
    # Verifico si hay valores nulos
    num_nulls = df[column].isnull().sum()
    print(f"\nNúmero de valores nulos: {num_nulls}")
    
    # Obtengo el tipo de datos de la columna
    dtype = df[column].dtype
    print(f"\nTipo de datos: {dtype}")
    
    # Si la columna es numérica, obtengo estadísticas descriptivas
    if pd.api.types.is_numeric_dtype(df[column]):
        print("\nEstadísticas descriptivas:")
        print(df[column].describe())
    
    # Si la columna es de tipo object (categórica), obtengo los valores únicos y sus conteos
    if dtype == 'object':
        print("\nValores únicos (top 5):")
        print(df[column].value_counts().head(5))
        
    # Si la columna contiene listas o diccionarios, analizo la longitud de los datos
    if df[column].apply(lambda x: isinstance(x, (list, dict))).any():
        print("\nLongitud de los datos (para listas y diccionarios):")
        print(df[column].apply(lambda x: len(x) if isinstance(x, (list, dict)) else None).describe())
    
    # Si la columna es de tipo int, str o datetime
    if np.issubdtype(dtype, np.integer):
        print("\nLa columna es de tipo entero.")
    elif np.issubdtype(dtype, np.datetime64):
        print("\nLa columna es de tipo datetime.")
    elif dtype == object:
        print("\nLa columna es de tipo string.")

# Uso de la función para una columna específica
df = df_user_reviews  # aquí coloco el nombre del df y su nombre
column = 'user_url'  # aquí coloco el nombre de la columna

analyze_individual_column(df, column)


Verifico duplicados.

In [ ]:
# Listo los IDs de url para verificar duplicados
user_url = ['http://steamcommunity.com/profiles/76561198061820192']  # Aquí puedes poner los IDs de usuario que quieras verificar

# Columnas a considerar al verificar duplicados
# Aquí puedes poner las columnas que quieras considerar al verificar duplicados
columns_to_consider = ['user_id', 'user_url', 'reviews']

# Para cada ID de usuario en la lista
for user_url in user_url:  
    # Crea una cadena de texto con la siguiente información:
    info = f"""
    Este bloque de código verifica si hay filas duplicadas para el ID de usuario 
    '{user_url}' en el DataFrame df_user_reviews. 
    Si hay filas duplicadas, las imprime.
    """
    # Imprimo la cadena de texto
    print(info)

    # Filtro el DataFrame para mostrar solo las filas donde 'user_url' es el ID de usuario actual
    df_id = df_user_reviews[df_user_reviews['user_url'] == user_url].copy()

    # Convierto las listas en la columna 'reviews' a una representación de cadena
    df_id['reviews'] = df_id['reviews'].apply(str)

    # Verifico si hay filas duplicadas, considerando solo las columnas especificadas
    duplicates = df_id.duplicated(subset=columns_to_consider)

    # Si hay filas duplicadas
    if duplicates.any():
        print(f"Duplicados para {user_url}:")  
        
        # Imprimo las filas duplicadas
        print(df_id[duplicates])
    else:
        # Imprimo un mensaje indicando que no se encontraron duplicados para el ID de usuario actual
        print(f"No se encontraron duplicados para {user_url}.")



Analizaré reviews.

In [4]:
# Muestra las primeras 5 filas de la columna 'reviews'
print(df_user_reviews['reviews'].head(10))



0    [{'funny': '', 'posted': 'Posted November 5, 2...
1    [{'funny': '', 'posted': 'Posted June 24, 2014...
2    [{'funny': '', 'posted': 'Posted February 3.',...
3    [{'funny': '', 'posted': 'Posted October 14, 2...
4    [{'funny': '3 people found this review funny',...
5    [{'funny': '', 'posted': 'Posted May 5, 2014.'...
6    [{'funny': '1 person found this review funny',...
7    [{'funny': '', 'posted': 'Posted July 24.', 'l...
8    [{'funny': '5 people found this review funny',...
9    [{'funny': '', 'posted': 'Posted June 16.', 'l...
Name: reviews, dtype: object


Desanido.

In [5]:
# Importo la librería necesaria
from pandas import json_normalize

# Itero sobre cada fila en 'reviews'
for index, row in df_user_reviews.iterrows():
    # Desanido la lista de diccionarios en 'reviews'
    df_temp = json_normalize(row['reviews'])
    # Renombro las columnas del DataFrame temporal
    df_temp.columns = [str(i+1) for i in range(len(df_temp.columns))]
    # Añado las columnas desanidadas al DataFrame existente
    df_user_reviews = pd.concat([df_user_reviews, df_temp], axis=1)

# Muestro las primeras filas del DataFrame modificado
df_user_reviews.head()


In [ ]:
# Muestra las primeras 5 filas del DataFrame
print(df_user_reviews.head())



<h5 style="color:blue">
Trabajaré con users items.

Verifico

In [ ]:
df_users_items

Verifico las columnas que contienen listas o diccionarios.

In [ ]:
# Importo librerías.
import pandas as pd

# Itero sobre cada columna
for column in df_users_items.columns:
    # Obtengo el primer valor no nulo
    non_null_values = df_users_items[column].dropna()
    if non_null_values.empty:
        print(f"La columna '{column}' solo contiene valores nulos.")
        continue
    first_non_null_value = non_null_values.iloc[0]

    # Compruebo si es una lista o un diccionario
    if isinstance(first_non_null_value, list):
        print(f"La columna '{column}' contiene una lista.")
    elif isinstance(first_non_null_value, dict):
        print(f"La columna '{column}' contiene un diccionario.")
    else:
        print(f"La columna '{column}' no contiene ni listas ni diccionarios.")

Analizo user id

In [ ]:
# Importo librerías
import pandas as pd
import numpy as np

def analyze_individual_column(df, column):
    """
    Esta función analiza una columna individual de un DataFrame de pandas.
    
    Parámetros:
    df (pandas.DataFrame): El DataFrame a analizar.
    column (str): El nombre de la columna a analizar.
    
    Retorna:
    None
    """
    print(f"\n\nAnalizando la columna '{column}'\n" + "-"*50)
    
    # Imprimo los primeros 20 valores
    print("\nPrimeros 20 valores:")
    print(df[column].head(20))
    
    # Verifico si hay valores nulos
    num_nulls = df[column].isnull().sum()
    print(f"\nNúmero de valores nulos: {num_nulls}")
    
    # Obtengo el tipo de datos de la columna
    dtype = df[column].dtype
    print(f"\nTipo de datos: {dtype}")
    
    # Si la columna es numérica, obtengo estadísticas descriptivas
    if pd.api.types.is_numeric_dtype(df[column]):
        print("\nEstadísticas descriptivas:")
        print(df[column].describe())
    
    # Si la columna es de tipo object (categórica), obtengo los valores únicos y sus conteos
    if dtype == 'object':
        print("\nValores únicos (top 5):")
        print(df[column].value_counts().head(5))
        
    # Si la columna contiene listas o diccionarios, analizo la longitud de los datos
    if df[column].apply(lambda x: isinstance(x, (list, dict))).any():
        print("\nLongitud de los datos (para listas y diccionarios):")
        print(df[column].apply(lambda x: len(x) if isinstance(x, (list, dict)) else None).describe())
    
    # Si la columna es de tipo int, str o datetime
    if np.issubdtype(dtype, np.integer):
        print("\nLa columna es de tipo entero.")
    elif np.issubdtype(dtype, np.datetime64):
        print("\nLa columna es de tipo datetime.")
    elif dtype == object:
        print("\nLa columna es de tipo string.")

# Uso de la función para una columna específica
df = df_users_items  # aquí coloco el nombre del df y su nombre
column = 'user_id'  # aquí coloco el nombre de la columna

analyze_individual_column(df, column)


Verifico duplicados.

In [ ]:
# Listo los IDs de id para verificar duplicados
user_id = ['76561198051777058']  # Aquí puedes poner los IDs de usuario que quieras verificar

# Columnas a considerar al verificar duplicados
# Aquí puedes poner las columnas que quieras considerar al verificar duplicados
columns_to_consider = ['user_id', 'item_acount', 'steam_id', 'user_url']

# Para cada ID de usuario en la lista
for user_id in user_id:  
    # Crea una cadena de texto con la siguiente información:
    info = f"""
    Este bloque de código verifica si hay filas duplicadas para el ID de usuario 
    '{user_id}' en el DataFrame df_user_reviews. 
    Si hay filas duplicadas, las imprime.
    """
    # Imprimo la cadena de texto
    print(info)

    # Filtro el DataFrame para mostrar solo las filas donde 'user_id' es el ID de usuario actual
    df_id = df_users_items[df_users_items['user_id'] == user_id].copy()

    # Convierto las listas en la columna 'items' a una representación de cadena
    df_id['items'] = df_id['items'].apply(str)

    # Verifico si hay filas duplicadas, considerando solo las columnas especificadas
    duplicates = df_id.duplicated(subset=columns_to_consider)

    # Si hay filas duplicadas
    if duplicates.any():
        print(f"Duplicados para {user_id}:")  
        
        # Imprimo las filas duplicadas
        print(df_id[duplicates])
    else:
        # Imprimo un mensaje indicando que no se encontraron duplicados para el ID de usuario actual
        print(f"No se encontraron duplicados para {user_id}.")



Analizo items count

In [ ]:
# Importo librerías
import pandas as pd
import numpy as np

def analyze_individual_column(df, column):
    """
    Esta función analiza una columna individual de un DataFrame de pandas.
    
    Parámetros:
    df (pandas.DataFrame): El DataFrame a analizar.
    column (str): El nombre de la columna a analizar.
    
    Retorna:
    None
    """
    print(f"\n\nAnalizando la columna '{column}'\n" + "-"*50)
    
    # Imprimo los primeros 20 valores
    print("\nPrimeros 20 valores:")
    print(df[column].head(20))
    
    # Verifico si hay valores nulos
    num_nulls = df[column].isnull().sum()
    print(f"\nNúmero de valores nulos: {num_nulls}")
    
    # Obtengo el tipo de datos de la columna
    dtype = df[column].dtype
    print(f"\nTipo de datos: {dtype}")
    
    # Si la columna es numérica, obtengo estadísticas descriptivas
    if pd.api.types.is_numeric_dtype(df[column]):
        print("\nEstadísticas descriptivas:")
        print(df[column].describe())
    
    # Si la columna es de tipo object (categórica), obtengo los valores únicos y sus conteos
    if dtype == 'object':
        print("\nValores únicos (top 5):")
        print(df[column].value_counts().head(5))
        
    # Si la columna contiene listas o diccionarios, analizo la longitud de los datos
    if df[column].apply(lambda x: isinstance(x, (list, dict))).any():
        print("\nLongitud de los datos (para listas y diccionarios):")
        print(df[column].apply(lambda x: len(x) if isinstance(x, (list, dict)) else None).describe())
    
    # Si la columna es de tipo int, str o datetime
    if np.issubdtype(dtype, np.integer):
        print("\nLa columna es de tipo entero.")
    elif np.issubdtype(dtype, np.datetime64):
        print("\nLa columna es de tipo datetime.")
    elif dtype == object:
        print("\nLa columna es de tipo string.")

# Uso de la función para una columna específica
df = df_users_items  # aquí coloco el nombre del df y su nombre
column = 'items_count'  # aquí coloco el nombre de la columna

analyze_individual_column(df, column)


Analizo steam id

In [ ]:
# Importo librerías
import pandas as pd
import numpy as np

def analyze_individual_column(df, column):
    """
    Esta función analiza una columna individual de un DataFrame de pandas.
    
    Parámetros:
    df (pandas.DataFrame): El DataFrame a analizar.
    column (str): El nombre de la columna a analizar.
    
    Retorna:
    None
    """
    print(f"\n\nAnalizando la columna '{column}'\n" + "-"*50)
    
    # Imprimo los primeros 20 valores
    print("\nPrimeros 20 valores:")
    print(df[column].head(20))
    
    # Verifico si hay valores nulos
    num_nulls = df[column].isnull().sum()
    print(f"\nNúmero de valores nulos: {num_nulls}")
    
    # Obtengo el tipo de datos de la columna
    dtype = df[column].dtype
    print(f"\nTipo de datos: {dtype}")
    
    # Si la columna es numérica, obtengo estadísticas descriptivas
    if pd.api.types.is_numeric_dtype(df[column]):
        print("\nEstadísticas descriptivas:")
        print(df[column].describe())
    
    # Si la columna es de tipo object (categórica), obtengo los valores únicos y sus conteos
    if dtype == 'object':
        print("\nValores únicos (top 5):")
        print(df[column].value_counts().head(5))
        
    # Si la columna contiene listas o diccionarios, analizo la longitud de los datos
    if df[column].apply(lambda x: isinstance(x, (list, dict))).any():
        print("\nLongitud de los datos (para listas y diccionarios):")
        print(df[column].apply(lambda x: len(x) if isinstance(x, (list, dict)) else None).describe())
    
    # Si la columna es de tipo int, str o datetime
    if np.issubdtype(dtype, np.integer):
        print("\nLa columna es de tipo entero.")
    elif np.issubdtype(dtype, np.datetime64):
        print("\nLa columna es de tipo datetime.")
    elif dtype == object:
        print("\nLa columna es de tipo string.")

# Uso de la función para una columna específica
df = df_users_items  # aquí coloco el nombre del df y su nombre
column = 'steam_id'  # aquí coloco el nombre de la columna

analyze_individual_column(df, column)


Analizo user url

In [ ]:
# Importo librerías
import pandas as pd
import numpy as np

def analyze_individual_column(df, column):
    """
    Esta función analiza una columna individual de un DataFrame de pandas.
    
    Parámetros:
    df (pandas.DataFrame): El DataFrame a analizar.
    column (str): El nombre de la columna a analizar.
    
    Retorna:
    None
    """
    print(f"\n\nAnalizando la columna '{column}'\n" + "-"*50)
    
    # Imprimo los primeros 20 valores
    print("\nPrimeros 20 valores:")
    print(df[column].head(20))
    
    # Verifico si hay valores nulos
    num_nulls = df[column].isnull().sum()
    print(f"\nNúmero de valores nulos: {num_nulls}")
    
    # Obtengo el tipo de datos de la columna
    dtype = df[column].dtype
    print(f"\nTipo de datos: {dtype}")
    
    # Si la columna es numérica, obtengo estadísticas descriptivas
    if pd.api.types.is_numeric_dtype(df[column]):
        print("\nEstadísticas descriptivas:")
        print(df[column].describe())
    
    # Si la columna es de tipo object (categórica), obtengo los valores únicos y sus conteos
    if dtype == 'object':
        print("\nValores únicos (top 5):")
        print(df[column].value_counts().head(5))
        
    # Si la columna contiene listas o diccionarios, analizo la longitud de los datos
    if df[column].apply(lambda x: isinstance(x, (list, dict))).any():
        print("\nLongitud de los datos (para listas y diccionarios):")
        print(df[column].apply(lambda x: len(x) if isinstance(x, (list, dict)) else None).describe())
    
    # Si la columna es de tipo int, str o datetime
    if np.issubdtype(dtype, np.integer):
        print("\nLa columna es de tipo entero.")
    elif np.issubdtype(dtype, np.datetime64):
        print("\nLa columna es de tipo datetime.")
    elif dtype == object:
        print("\nLa columna es de tipo string.")

# Uso de la función para una columna específica
df = df_users_items  # aquí coloco el nombre del df y su nombre
column = 'user_url'  # aquí coloco el nombre de la columna

analyze_individual_column(df, column)


Analizo items primero desanido.

In [ ]:
# Importo librerías
import pandas as pd
import numpy as np

def analyze_individual_column(df, column):
    """
    Esta función analiza una columna individual de un DataFrame de pandas.
    
    Parámetros:
    df (pandas.DataFrame): El DataFrame a analizar.
    column (str): El nombre de la columna a analizar.
    
    Retorna:
    None
    """
    print(f"\n\nAnalizando la columna '{column}'\n" + "-"*50)
    
    # Imprimo los primeros 20 valores
    print("\nPrimeros 20 valores:")
    print(df[column].head(20))
    
    # Verifico si hay valores nulos
    num_nulls = df[column].isnull().sum()
    print(f"\nNúmero de valores nulos: {num_nulls}")
    
    # Obtengo el tipo de datos de la columna
    dtype = df[column].dtype
    print(f"\nTipo de datos: {dtype}")
    
    # Si la columna es numérica, obtengo estadísticas descriptivas
    if pd.api.types.is_numeric_dtype(df[column]):
        print("\nEstadísticas descriptivas:")
        print(df[column].describe())
    
    # Si la columna es de tipo object (categórica), obtengo los valores únicos y sus conteos
    if dtype == 'object':
        print("\nValores únicos (top 5):")
        print(df[column].value_counts().head(5))
        
    # Si la columna contiene listas o diccionarios, analizo la longitud de los datos
    if df[column].apply(lambda x: isinstance(x, (list, dict))).any():
        print("\nLongitud de los datos (para listas y diccionarios):")
        print(df[column].apply(lambda x: len(x) if isinstance(x, (list, dict)) else None).describe())
    
    # Si la columna es de tipo int, str o datetime
    if np.issubdtype(dtype, np.integer):
        print("\nLa columna es de tipo entero.")
    elif np.issubdtype(dtype, np.datetime64):
        print("\nLa columna es de tipo datetime.")
    elif dtype == object:
        print("\nLa columna es de tipo string.")

# Uso de la función para una columna específica
df = df_users_items  # aquí coloco el nombre del df y su nombre
column = 'items'  # aquí coloco el nombre de la columna

analyze_individual_column(df, column)
